In [ ]:
%pip install --upgrade openai
%pip install claudette
%pip install python-dotenv
%pip install -U bitsandbytes
%pip install optimum
%pip install auto-gptq
%pip install Wikipedia-API

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 34.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33107 sha256=0ea7cbc5b688d5dd17f575f91a13e86b362e276ad0d27b22dbf2263bff30553e
  Stored in directory: /home/zeus/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 265.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 221.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 133.5 MB/s e

In [ ]:
import tokenize, ast
from io import BytesIO
import os

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch
import torch

In [ ]:
from huggingface_hub import login
login()

In [ ]:
mn = "meta-llama/Llama-2-7b-hf"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [ ]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,quantization_config=quantization_config)

In [ ]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Silver Rubanza is a "
toks = tokr(prompt, return_tensors="pt")

In [ ]:
toks

In [ ]:
tokr.batch_decode(toks['input_ids'])

In [ ]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

In [ ]:
tokr.batch_decode(res)

In [ ]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,torch_dtype=torch.bfloat16)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

In [ ]:
tokr.batch_decode(res)

In [ ]:
def gen(p, maxlen=15, sample=True):
  toks = tokr(p,return_tensors="pt")
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
  return tokr.batch_decode(res)

In [ ]:
%%time
gen(prompt,50)

## Other Models

### Stable Beluga

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "### System:\n You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. . \n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys):
    return f"{syst} ### User: {user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Silver Rubanza?"

In [ ]:
%%time
gen(mk_prompt(ques), 150)

### Open Orca / Playtpus 2

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user):
    return f"### Instruction: {user}\n\n ### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques),150)

## Retrival Augmented Generation

In [ ]:
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
jh_page

In [ ]:
print(jh_page[:500])

In [ ]:
print(type(jh_page))

In [ ]:
sr_page = 'Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work'

In [ ]:
len(jh_page.split()),len(sr_page.split())

In [ ]:
ques_ctx = f"""Answer the question with the help of the provided context.

## Context
{sr_page}

## Question
{ques}

"""

In [ ]:
res = gen(mk_prompt(ques_ctx),300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])

In [ ]:
%pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
sr_page

In [ ]:
sr = sr_page.split('\n\n')[0]
print(sr)

In [ ]:
q_emb,jh_emb,sr_emb = emb_model.encode([ques,jh,sr], convert_to_tensor=True)

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, sr_emb, dim=0)

## Finetuning

import datasets

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_medical_dialogue_v2')

In [ ]:
trn = ds['train']
trn[3]

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])